In [1]:
import os
import sys
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies
import originalpath
sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/9casminer_v2/')
import locus

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/"
outdir="/shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/9output/"+feature+"/"

In [7]:
cas9dataset=pd.read_csv(datadir+"5caslocitable/known_Cas9_variants_table.csv", index_col=0)

In [5]:
chosen_IDs_file=open(datadir+"6mining/"+feature+"/97_clusters/9aCsfalf1musi_centroid97_all.faa")
chosen_IDs=[]
for line in chosen_IDs_file.readlines():
    if line.startswith(">"):
        current_ID=line.strip("\n").split("__")[1] # take the first seqID of that fasta name
        chosen_IDs.append(current_ID)

In [18]:
# subset cas9dataset with chosen sequences
cas9dataset=cas9dataset[cas9dataset["Seq ID"].isin(chosen_IDs)]
print(cas9dataset.shape,cas9dataset.columns)

(49, 17) Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy', 'prokka_cas9'],
      dtype='object')


## Data sorting

#### 1. Sort by length

In [32]:
cas9dataset["Sequence length"]=cas9dataset["Seq"].str.count("") #count method adds 1 digit, 
# so we must subtract it
cas9dataset["Sequence length"]=cas9dataset["Sequence length"]-1

In [33]:
#es
len(cas9dataset.loc[618]["Seq"])==cas9dataset.loc[618]["Sequence length"]

True

In [43]:
cas9dataset.sort_values(by=["Sequence length"], inplace=True)

In [51]:
# rearrange order of columns

cas9dataset=cas9dataset[['Seq ID', 'Sequence length', 'Contig', 'uSGB', 'SGB ID', 'Level of estimated taxonomy', 'Seq Description', 'Seq', 'Genome Name', 'Study','Sample Name', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR','prokka_cas1', 'prokka_cas2','Estimated taxonomy', 'prokka_cas9']]

In [52]:
cas9dataset

,Seq ID,Sequence length,Contig,uSGB,SGB ID,Level of estimated taxonomy,Seq Description,Seq,Genome Name,Study,Sample Name,Pos,pilercr_CRISPR,minced_CRISPR,prokka_cas1,prokka_cas2,Estimated taxonomy,prokka_cas9
11090,KJDMGLGH_01090,949,NODE_114_length_42672_cov_11.9569,Yes,15299,Genus,KJDMGLGH_01090 CRISPR-associated endonuclease ...,MKHPYGIGLDIGIASVGWAVMALNENAEPYGLIRCGSRIFDKAEQP...,KarlssonFH_2013__S235__bin.6,KarlssonFH_2013,S235,12906 15755,"[('ID=CRISPR1', '11276', '11')]","[('.', '10513', '11276')]","[('ID=KJDMGLGH_01088', '11638', '12519')]","[('ID=KJDMGLGH_01087', '11325', '11633')]",k__Bacteria|p__Firmicutes|c__Clostridia|o__Clo...,"('ID=KJDMGLGH_01090', '12906', '15755')"
19055,AOPFDEBE_00006,978,NODE_411_length_19879_cov_2.8009,Yes,17319,Other,AOPFDEBE_00006 CRISPR-associated endonuclease ...,MSLIFGFDLGIASIGWAAVKMSDPTASDTDAAAPDNRPVGEILGAG...,Obregon-TitoAJ_2015__SM31__bin.19,Obregon-TitoAJ_2015,SM31,6519 9455,"[('ID=CRISPR1', '11227', '6')]","[('.', '10862', '12283')]","[('ID=AOPFDEBE_00007', '9510', '10424')]","[('ID=AOPFDEBE_00008', '10453', '10758')]",-,"('ID=AOPFDEBE_00006', '6519', '9455')"
2351,FLJLKKHA_00012,979,NODE_49_length_176785_cov_12.2725,Yes,9311,Family,FLJLKKHA_00012 CRISPR-associated endonuclease ...,MIFGIASLGWAVVDFDDAADPENNIYPTGKIVKSGVRCFPVAENPK...,BritoIL_2016__M1.29.ST__bin.30,BritoIL_2016,M1.29.ST,15614 18553,"[('ID=CRISPR1', '14320', '10')]","[('.', '13625', '14320')]","[('ID=FLJLKKHA_00011', '14715', '15614')]","[('ID=FLJLKKHA_00010', '14378', '14707')]",k__Bacteria|p__Proteobacteria|c__Alphaproteoba...,"('ID=FLJLKKHA_00012', '15614', '18553')"
14876,NCCHGPDI_00805,995,NODE_7099_length_5039_cov_5.72392,Yes,15286,Family,NCCHGPDI_00805 CRISPR-associated endonuclease ...,MEKNGLPGKAEIEQAVQAGHLPDVYALRVQALDGPVTALDFARILL...,LiJ_2014__O2.UC31-1__bin.35,LiJ_2014,O2.UC31-1,41 3028,"[('ID=CRISPR1', '4640', '6')]","[('.', '4275', '4640')]","[('ID=NCCHGPDI_00806', '3032', '3913')]","[('ID=NCCHGPDI_00807', '4017', '4226')]",k__Bacteria|p__Firmicutes|c__Clostridia|o__Clo...,"('ID=NCCHGPDI_00805', '41', '3028')"
3910,DCEHCAPA_00438,997,NODE_473_length_19808_cov_4.46626,Yes,17319,Other,DCEHCAPA_00438 CRISPR-associated endonuclease ...,MSLIFGFDLGIASIGWAAVKMSDPTASDTDAAAPDNRPVGEILGAG...,CM_madagascar__A39_01_1FE__bin.31,CM_madagascar,A39_01_1FE,11892 14885,"[('ID=CRISPR1', '17383', '17')]","[('.', '16292', '17383')]","[('ID=DCEHCAPA_00439', '14940', '15854')]","[('ID=DCEHCAPA_00440', '15883', '16188')]",-,"('ID=DCEHCAPA_00438', '11892', '14885')"
6607,MKCMDOMJ_00229,1000,NODE_219_length_70277_cov_6.1352,Yes,9243,Other,MKCMDOMJ_00229 CRISPR-associated endonuclease ...,MQDSVSKMKYRLGIDLGTTSLGWAMLRLDEQNEPYAVIRTGVRIFN...,CosteaPI_2017__SID713B076-11-0-0__bin.41,CosteaPI_2017,SID713B076-11-0-0,72 3074,"[('ID=CRISPR1', '5081', '11')]","[('.', '4390', '5081')]","[('ID=MKCMDOMJ_00230', '3071', '4012')]","[('ID=MKCMDOMJ_00231', '4017', '4322')]",-,"('ID=MKCMDOMJ_00229', '72', '3074')"
31643,BNKOANIA_00910,1002,NODE_509_length_33310_cov_9.9946_ID_70650,Yes,9243,Other,BNKOANIA_00910 CRISPR-associated endonuclease ...,MKMQDSVSKMKYRLGIDLGTTSLGWAMLRLDEQNEPYAVIRAGVRI...,ZeeviD_2015__PNP_Main_609__bin.14,ZeeviD_2015,PNP_Main_609,1552 4560,"[('ID=CRISPR1', '.', '4')]","[('.', '3', '235')]","[('ID=BNKOANIA_00909', '614', '1555')]","[('ID=BNKOANIA_00908', '304', '630')]",-,"('ID=BNKOANIA_00910', '1552', '4560')"
11269,LJKGNKPJ_00564,1002,NODE_211_length_45519_cov_7.71687,Yes,14454,Genus,LJKGNKPJ_00564 CRISPR-associated endonuclease ...,MEITINREIGKLGLPRHLVLGMDPGIASCGFALIDTANHEILDLGV...,KarlssonFH_2013__S420__bin.8,KarlssonFH_2013,S420,9720 12728,"[('ID=CRISPR1', '13885', '13')]","[('.', '12991', '13885')]","[('ID=LJKGNKPJ_00566', '14325', '15215')]","[('ID=LJKGNKPJ_00567', '15225', '15542')]",k__Bacteria|p__Actinobacteria|c__Coriobacterii...,"('ID=LJKGNKPJ_00564', '9720', '12728')"
831,DFMBJCCJ_00626,1003,NODE_290_length_27586_cov_10.0642,Yes,14454,Genus,DFMBJCCJ_00626 CRISPR-associated endonuclease ...,MEITINREIGKLGLPRHLVLGMDPGIASCGFAL

In [66]:
# 2 contig coverage
coverage=[x[5] for i,x in cas9dataset["Contig"].str.split("_").iteritems()]
cas9dataset["Coverage"]=coverage
#reorder to have contig at the beginning
cas9dataset=cas9dataset[['Seq ID', 'Sequence length', 'Coverage', 'uSGB', 'SGB ID', 'Level of estimated taxonomy', 'Seq Description', 'Seq', 'Genome Name', 'Study','Sample Name', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR','prokka_cas1', 'prokka_cas2','Estimated taxonomy', 'prokka_cas9','Contig']]

/home/lorenzo.signorini/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
cas9dataset.sort_values(by=["Sequence length","Coverage"], ascending=(True,False), inplace=True)
cas9dataset


,Seq ID,Sequence length,Coverage,uSGB,SGB ID,Level of estimated taxonomy,Seq Description,Seq,Genome Name,Study,Sample Name,Pos,pilercr_CRISPR,minced_CRISPR,prokka_cas1,prokka_cas2,Estimated taxonomy,prokka_cas9,Contig
11090,KJDMGLGH_01090,949,11.9569,Yes,15299,Genus,KJDMGLGH_01090 CRISPR-associated endonuclease ...,MKHPYGIGLDIGIASVGWAVMALNENAEPYGLIRCGSRIFDKAEQP...,KarlssonFH_2013__S235__bin.6,KarlssonFH_2013,S235,12906 15755,"[('ID=CRISPR1', '11276', '11')]","[('.', '10513', '11276')]","[('ID=KJDMGLGH_01088', '11638', '12519')]","[('ID=KJDMGLGH_01087', '11325', '11633')]",k__Bacteria|p__Firmicutes|c__Clostridia|o__Clo...,"('ID=KJDMGLGH_01090', '12906', '15755')",NODE_114_length_42672_cov_11.9569
19055,AOPFDEBE_00006,978,2.8009,Yes,17319,Other,AOPFDEBE_00006 CRISPR-associated endonuclease ...,MSLIFGFDLGIASIGWAAVKMSDPTASDTDAAAPDNRPVGEILGAG...,Obregon-TitoAJ_2015__SM31__bin.19,Obregon-TitoAJ_2015,SM31,6519 9455,"[('ID=CRISPR1', '11227', '6')]","[('.', '10862', '12283')]","[('ID=AOPFDEBE_00007', '9510', '10424')]","[('ID=AOPFDEBE_00008', '10453', '10758')]",-,"('ID=AOPFDEBE_00006', '6519', '9455')",NODE_411_length_19879_cov_2.8009
2351,FLJLKKHA_00012,979,12.2725,Yes,9311,Family,FLJLKKHA_00012 CRISPR-associated endonuclease ...,MIFGIASLGWAVVDFDDAADPENNIYPTGKIVKSGVRCFPVAENPK...,BritoIL_2016__M1.29.ST__bin.30,BritoIL_2016,M1.29.ST,15614 18553,"[('ID=CRISPR1', '14320', '10')]","[('.', '13625', '14320')]","[('ID=FLJLKKHA_00011', '14715', '15614')]","[('ID=FLJLKKHA_00010', '14378', '14707')]",k__Bacteria|p__Proteobacteria|c__Alphaproteoba...,"('ID=FLJLKKHA_00012', '15614', '18553')",NODE_49_length_176785_cov_12.2725
14876,NCCHGPDI_00805,995,5.72392,Yes,15286,Family,NCCHGPDI_00805 CRISPR-associated endonuclease ...,MEKNGLPGKAEIEQAVQAGHLPDVYALRVQALDGPVTALDFARILL...,LiJ_2014__O2.UC31-1__bin.35,LiJ_2014,O2.UC31-1,41 3028,"[('ID=CRISPR1', '4640', '6')]","[('.', '4275', '4640')]","[('ID=NCCHGPDI_00806', '3032', '3913')]","[('ID=NCCHGPDI_00807', '4017', '4226')]",k__Bacteria|p__Firmicutes|c__Clostridia|o__Clo...,"('ID=NCCHGPDI_00805', '41', '3028')",NODE_7099_length_5039_cov_5.72392
3910,DCEHCAPA_00438,997,4.46626,Yes,17319,Other,DCEHCAPA_00438 CRISPR-associated endonuclease ...,MSLIFGFDLGIASIGWAAVKMSDPTASDTDAAAPDNRPVGEILGAG...,CM_madagascar__A39_01_1FE__bin.31,CM_madagascar,A39_01_1FE,11892 14885,"[('ID=CRISPR1', '17383', '17')]","[('.', '16292', '17383')]","[('ID=DCEHCAPA_00439', '14940', '15854')]","[('ID=DCEHCAPA_00440', '15883', '16188')]",-,"('ID=DCEHCAPA_00438', '11892', '14885')",NODE_473_length_19808_cov_4.46626
6607,MKCMDOMJ_00229,1000,6.1352,Yes,9243,Other,MKCMDOMJ_00229 CRISPR-associated endonuclease ...,MQDSVSKMKYRLGIDLGTTSLGWAMLRLDEQNEPYAVIRTGVRIFN...,CosteaPI_2017__SID713B076-11-0-0__bin.41,CosteaPI_2017,SID713B076-11-0-0,72 3074,"[('ID=CRISPR1', '5081', '11')]","[('.', '4390', '5081')]","[('ID=MKCMDOMJ_00230', '3071', '4012')]","[('ID=MKCMDOMJ_00231', '4017', '4322')]",-,"('ID=MKCMDOMJ_00229', '72', '3074')",NODE_219_length_70277_cov_6.1352
31643,BNKOANIA_00910,1002,9.9946,Yes,9243,Other,BNKOANIA_00910 CRISPR-associated endonuclease ...,MKMQDSVSKMKYRLGIDLGTTSLGWAMLRLDEQNEPYAVIRAGVRI...,ZeeviD_2015__PNP_Main_609__bin.14,ZeeviD_2015,PNP_Main_609,1552 4560,"[('ID=CRISPR1', '.', '4')]","[('.', '3', '235')]","[('ID=BNKOANIA_00909', '614', '1555')]","[('ID=BNKOANIA_00908', '304', '630')]",-,"('ID=BNKOANIA_00910', '1552', '4560')",NODE_509_length_33310_cov_9.9946_ID_70650
11269,LJKGNKPJ_00564,1002,7.71687,Yes,14454,Genus,LJKGNKPJ_00564 CRISPR-associated endonuclease ...,MEITINREIGKLGLPRHLVLGMDPGIASCGFALIDTANHEILDLGV...,KarlssonFH_2013__S420__bin.8,KarlssonFH_2013,S420,9720 12728,"[('ID=CRISPR1', '13885', '13')]","[('.', '12991', '13885')]","[('ID=LJKGNKPJ_00566', '14325', '15215')]","[('ID=LJKGNKPJ_00567', '15225', '15542')]",k__Bacteria|p__Actinobacteria|c__Coriobacterii...,"('ID=LJKGNKPJ_00564', '9720', '12728')",NODE_211_length_45519_cov_7.71687
4436,LFLJBGAC_00085,1003,20.0924,Yes,9281,Family,LFLJBGAC_00085 CRISPR-associated endon